In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd 
import numpy as np
from itertools import product, permutations

import numba
from numba import njit,vectorize, jit
from tqdm import tqdm

import Estimators

In [10]:
d = 2
c = 0

In [53]:
#@njit
sig2 = 1
@njit
def GaussianKernel(x,y):
    return np.exp(-np.linalg.norm(x-y)**2/(2*sig2))

@njit
def PolynomialKernel(x,y): 
    return (x.dot(y)+c)**d
@njit
def LinearKernel(x,y): 
    return x.dot(y)

@njit
def Laplace_kernel(x, y, gamma=1):
    return 0.5 * np.exp(-gamma * np.linalg.norm(x-y))

In [4]:
def importation(): 
    X_train = []
    X_test = []
    Y_train = []
    for i in range(3): 
        xtrain = pd.read_csv('data/Xtr'+str(i)+'.csv',delimiter= ',', header= None)
        xtrain = xtrain.iloc[1:,1].to_numpy()
        X_train.append(xtrain)
    
        xtest = pd.read_csv('data/Xte'+str(i)+'.csv',delimiter= ',', header= None)
        xtest = xtest.iloc[1:,1]
        X_test.append(xtest)
    
        Y_train.append(pd.read_csv('data/Ytr'+str(i)+'.csv',delimiter= ',')['Bound'].to_numpy())
    return X_train,X_test,Y_train

X_train,X_test,Y_train = importation()

In [5]:
X_mat100_0 = pd.read_csv('data/Xtr0_mat100.csv',delimiter= ' ', header= None).values
Y_mat100_0 = pd.read_csv('data/Ytr0.csv',delimiter= ',')['Bound'].to_numpy()

In [6]:
X_traintest = []
for i in range(3): 
    X_traintest.append(np.concatenate((X_train[i],X_test[i])))


In [7]:
def to_Kernel_train(X, Kernel): 
    length = X.shape[0]
    mat_K = np.zeros((length,length))
    for i in range(length):
        x_i = np.squeeze(X[i])
        for j in range(i,length): 
            x_j = np.squeeze(X[j])
            value = Kernel(x_i,x_j)
            mat_K[i,j] = value
            mat_K[j,i] = value 
    return mat_K

#@njit 
def to_Kernel_test(Xtrain,Xtest,Kernel):
    length_train = Xtrain.shape[0]
    length_test = Xtest.shape[0]
    bimat_K = np.zeros((length_train,length_test))
    for i in range(length_train):
        x_i = np.squeeze(Xtrain[i])
        for j in range(length_test): 
            x_j = np.squeeze(Xtest[j])
            value = Kernel(x_i,x_j)
            bimat_K[i,j] = value
    return bimat_K

In [8]:
# Compute phi for Spectrum kernel
def phi_spectrum(x,k,U):
    """U: list of the sequences of size k to look for in x"""
    phi_spec = np.zeros(len(U))
    for j, u in enumerate(U):
        for i in range(len(x)-k+1):
            if x[i:i+k] == u:
                phi_spec[j] +=1
    return phi_spec

#Compute phi for Substring kernel
def l(i):
    return i[-1] - i[0] + 1

def I(k,n):
    I = set()
    for ele in permutations(range(0,n),k):
        I.add(tuple(sorted(list(ele))))
    return list(I)

def phi_substring(x,k,U,lamb=0.5): #fonctionne (testé avec l'exemple page 392)
    """U: list of the sequences of size k to look for in x"""
    phi = np.zeros(len(U))
    I_kn = I(k,len(x))
    for j, u in enumerate(U):
        for i in I_kn:
            x_i = "".join([x[idx]  for idx in i])
            if x_i==u:
                phi[j] += lamb**l(i)
    return phi


def make_dict_phi(X, phi,k):
    U = [''.join(letter) for letter in product('ACGT', repeat=k)]
    phi_dict = {seq:phi(seq,k,U) for seq in tqdm(X)}
    return phi_dict
    

# Compute K(x_i,x) 
def K_substring_2(x, y):
    """
    Compute K(x_i,x) with K a string kernel
    
    Parameters:
    Ne prend pas en argument X_train mais seulement phi_X_train(qui a été calculé précédement)
    et i .
    
    phi_X_train: list of phi(X_train) which is also needed to construct the matrix K 
    phi: the function phi
    k: hyperparameter of the spectrum kernel (size of sequences in U)
    """
    value = np.sum([dict_sub_traintest_2[str(x)] * dict_sub_traintest_2[str(y)]])  
    return value

def K_substring_4(x,y): 
    value = np.sum([dict_sub_traintest_4[str(x)] * dict_sub_traintest_4[str(y)]])  
    return value
    

def K_spectrum_3(x,y): 
    value = np.sum([dict_spectrum_traintest_3[str(x)] * dict_spectrum_traintest_3[str(y)]])  
    return value
    
def K_spectrum_4(x,y): 
    value = np.sum([dict_spectrum_traintest_4[str(x)] * dict_spectrum_traintest_4[str(y)]])  
    return value

def K_spectrum_5(x,y): 
    value = np.sum([dict_spectrum_traintest_5[str(x)] * dict_spectrum_traintest_5[str(y)]])  
    return value




In [64]:
##CONSTRUCTION DES DICTIONNAIRES. On fait ça pour eviter de les recalculer à chaque fois

dict_spectrum_traintest_3 = make_dict_phi(X_traintest[0],phi_spectrum,3)

#dict_spectrum_traintest_4 = make_dict_phi(X_traintest[0],phi_spectrum,4)

#dict_spectrum_traintest_5 = make_dict_phi(X_traintest[0],phi_spectrum,5)

#dict_sub_traintest_2 = make_dict_phi(X_traintest[0], phi_substring,2)

#dict_sub_traintest_3 = make_dict_phi(X_traintest[0], phi_substring,3)

100%|██████████| 3000/3000 [00:08<00:00, 346.01it/s]


In [21]:
Kernel_train_Gaussian = to_Kernel_train(X_mat100_0,GaussianKernel)
Kernel_test_Gaussian = to_Kernel_test(X_mat100_0,X_mat100_0,GaussianKernel)

In [11]:
Kernel_train_Linear = to_Kernel_train(X_mat100_0,LinearKernel)
Kernel_test_Linear = to_Kernel_test(X_mat100_0,X_mat100_0,LinearKernel)

<ipython-input-9-879bc3027521>:11: NumbaPerformanceWarning: np.dot() is faster on contiguous arrays, called on (array(float64, 1d, A), array(float64, 1d, A))
  return x.dot(y)


In [70]:
Kernel_train_spectrum_3 = to_Kernel_train(X_train[0],K_spectrum_3)
Kernel_test_spectrum_3 = to_Kernel_test(X_train[0], X_train[0],K_spectrum_3)

In [59]:
def try_Kernel(Kernel,X,Kernel_train = 0,Kernel_test = 0,pre_computed = False): 
    if pre_computed == False : 
        Kernel_train = to_Kernel_train(X,Kernel)
        Kernel_test = to_Kernel_test(X,X,Kernel)
    regressor = Estimators.KRR(Kernel = Kernel)
    regressor.fit(Kernel_train,Y_train[0])
    pred = regressor.predict(Kernel_test)
    print('predictions-true : ', (pred-Y_train[0])[0:100])
    print('predictions : ', pred[0:100]*1)
    print('true : ', Y_train[0][0:100])
    print('SCORE : ', 1-np.sum(np.abs(pred-Y_train[0]))/pred.shape[0])

In [60]:
try_Kernel(GaussianKernel,X_mat100_0,Kernel_train_Gaussian,Kernel_test_Gaussian, pre_computed = True)

predictions-true :  [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
predictions :  [0 1 1 1 1 0 0 0 0 1 0 1 1 0 1 0 1 1 1 1 0 1 1 1 0 1 1 1 0 0 0 1 1 1 1 0 0
 0 0 1 1 0 0 0 0 0 0 1 0 0 1 0 1 0 0 1 1 1 1 0 1 0 0 0 0 0 0 0 0 1 0 1 0 1
 0 1 0 0 1 1 1 0 1 0 1 0 0 0 0 1 1 1 0 0 0 1 0 1 1 0]
true :  [0 1 1 1 1 0 0 0 0 1 0 1 1 0 1 0 1 1 1 1 0 1 1 1 0 1 1 1 0 0 0 1 1 1 1 0 0
 0 0 1 1 0 0 0 0 0 0 1 0 0 1 0 1 0 0 1 1 1 1 0 1 0 0 0 0 0 0 0 0 1 0 1 0 1
 0 1 0 0 1 1 1 0 1 0 1 0 0 0 0 1 1 1 0 0 0 1 0 1 1 0]
SCORE :  0.9955


In [61]:
try_Kernel(PolynomialKernel,X_mat100_0)

predictions-true :  [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
predictions :  [0 1 1 1 1 0 0 0 0 1 0 1 1 0 1 0 1 1 1 1 0 1 1 1 0 1 1 1 0 0 0 1 1 1 1 0 0
 0 0 1 1 0 0 0 0 0 0 1 0 0 1 0 1 0 0 1 1 1 1 0 1 0 0 0 0 0 0 0 0 1 0 1 0 1
 0 1 0 0 1 1 1 0 1 0 1 0 0 0 0 1 1 1 0 0 0 1 0 1 1 0]
true :  [0 1 1 1 1 0 0 0 0 1 0 1 1 0 1 0 1 1 1 1 0 1 1 1 0 1 1 1 0 0 0 1 1 1 1 0 0
 0 0 1 1 0 0 0 0 0 0 1 0 0 1 0 1 0 0 1 1 1 1 0 1 0 0 0 0 0 0 0 0 1 0 1 0 1
 0 1 0 0 1 1 1 0 1 0 1 0 0 0 0 1 1 1 0 0 0 1 0 1 1 0]
SCORE :  0.9985


In [62]:
try_Kernel(LinearKernel,X_mat100_0)

predictions-true :  [ 1 -1 -1  0  0  0  1  1  1  0  1  0 -1  0  0  0  0  0  0  0  0  0 -1  0
  0 -1  0  0  1  0  0  0  0  0  0  1  1  1  0 -1 -1  0  0  1  0  0  1  0
  1  0 -1  0 -1  1  1 -1 -1 -1  0  0  0  0  1  0  0  0  1  0  1  0  1  0
  0  0  0  0  1  1  0  0  0  0  0  0  0  1  0  0  0 -1 -1  0  0  0  0  0
  0  0 -1  0]
predictions :  [1 0 0 1 1 0 1 1 1 1 1 1 0 0 1 0 1 1 1 1 0 1 0 1 0 0 1 1 1 0 0 1 1 1 1 1 1
 1 0 0 0 0 0 1 0 0 1 1 1 0 0 0 0 1 1 0 0 0 1 0 1 0 1 0 0 0 1 0 1 1 1 1 0 1
 0 1 1 1 1 1 1 0 1 0 1 1 0 0 0 0 0 1 0 0 0 1 0 1 0 0]
true :  [0 1 1 1 1 0 0 0 0 1 0 1 1 0 1 0 1 1 1 1 0 1 1 1 0 1 1 1 0 0 0 1 1 1 1 0 0
 0 0 1 1 0 0 0 0 0 0 1 0 0 1 0 1 0 0 1 1 1 1 0 1 0 0 0 0 0 0 0 0 1 0 1 0 1
 0 1 0 0 1 1 1 0 1 0 1 0 0 0 0 1 1 1 0 0 0 1 0 1 1 0]
SCORE :  0.652


In [63]:
try_Kernel(Laplace_kernel,X_mat100_0)

predictions-true :  [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
predictions :  [0 1 1 1 1 0 0 0 0 1 0 1 1 0 1 0 1 1 1 1 0 1 1 1 0 1 1 1 0 0 0 1 1 1 1 0 0
 0 0 1 1 0 0 0 0 0 0 1 0 0 1 0 1 0 0 1 1 1 1 0 1 0 0 0 0 0 0 0 0 1 0 1 0 1
 0 1 0 0 1 1 1 0 1 0 1 0 0 0 0 1 1 1 0 0 0 1 0 1 1 0]
true :  [0 1 1 1 1 0 0 0 0 1 0 1 1 0 1 0 1 1 1 1 0 1 1 1 0 1 1 1 0 0 0 1 1 1 1 0 0
 0 0 1 1 0 0 0 0 0 0 1 0 0 1 0 1 0 0 1 1 1 1 0 1 0 0 0 0 0 0 0 0 1 0 1 0 1
 0 1 0 0 1 1 1 0 1 0 1 0 0 0 0 1 1 1 0 0 0 1 0 1 1 0]
SCORE :  1.0


In [65]:
try_Kernel(K_spectrum_3, X_train[0])

predictions-true :  [ 0  0 -1  0  0  0  0  1  0  0  0 -1  0  1 -1  0  0 -1 -1  0  1 -1  0 -1
  0 -1 -1  0  0  0  0 -1  0 -1 -1  1  0  1  0 -1  0  1  1  1  0  0  0  0
  1  0  0  0  0  0  1 -1 -1  0  0  1  0  0  0  0  0  0  1  1  1 -1  0  0
  0 -1  1 -1  0  0  0  0  0  0 -1  0  0  1  0  0  0 -1  0  0  0  0  0  0
  0 -1  0  0]
predictions :  [0 1 0 1 1 0 0 1 0 1 0 0 1 1 0 0 1 0 0 1 1 0 1 0 0 0 0 1 0 0 0 0 1 0 0 1 0
 1 0 0 1 1 1 1 0 0 0 1 1 0 1 0 1 0 1 0 0 1 1 1 1 0 0 0 0 0 1 1 1 0 0 1 0 0
 1 0 0 0 1 1 1 0 0 0 1 1 0 0 0 0 1 1 0 0 0 1 0 0 1 0]
true :  [0 1 1 1 1 0 0 0 0 1 0 1 1 0 1 0 1 1 1 1 0 1 1 1 0 1 1 1 0 0 0 1 1 1 1 0 0
 0 0 1 1 0 0 0 0 0 0 1 0 0 1 0 1 0 0 1 1 1 1 0 1 0 0 0 0 0 0 0 0 1 0 1 0 1
 0 1 0 0 1 1 1 0 1 0 1 0 0 0 0 1 1 1 0 0 0 1 0 1 1 0]
SCORE :  0.6165


In [67]:
regressor_Gaussian = Estimators.KRR(Kernel = GaussianKernel)
regressor_Gaussian.cross_val(Kernel_train_Gaussian,Y_train[0], 5)

5it [00:00, 12.10it/s]

Score :  0.581


0.419

In [101]:
regressor_Linear = Estimators.KRR(Kernel = LinearKernel,lam = 1e-12)
regressor_Linear.cross_val(Kernel_train_Linear,Y_train[0],5)

5it [00:00, 10.90it/s]

Score :  0.5954999999999999


0.4045

In [80]:
regressor_Laplace = Estimators.KRR(Kernel = Laplace_kernel)
regressor_Linear.cross_val(Kernel_train_Linear,Y_train[0],5)

5it [00:00, 12.84it/s]

Score :  0.5895


0.4105

In [79]:
regressor_spectrum = Estimators.KRR(Kernel = K_spectrum_3,lam = 0.002)
regressor_spectrum.cross_val(Kernel_train_spectrum_3,Y_train[0],5)

5it [00:04,  1.22it/s]

Score :  0.581


0.419

In [75]:
for i in range(10): 
    print('lambda = ', 10**(-i+5))
    regressor_spectrum = Estimators.KRR(Kernel = K_spectrum_3,lam = 10**(-i+5))
    regressor_spectrum.cross_val(Kernel_train_spectrum_3,Y_train[0],5)
    

0it [00:00, ?it/s]

lambda =  100000


5it [00:04,  1.13it/s]
0it [00:00, ?it/s]

Score :  0.519
lambda =  10000


5it [00:00, 11.67it/s]
0it [00:00, ?it/s]

...

Score :  0.5745
lambda =  0.001


5it [00:00,  8.12it/s]


Score :  0.5740000000000001
lambda =  0.0001


5it [00:00,  6.33it/s]

Score :  0.5835


In [85]:
for i in range(10): 
    print('lambda = ', 10**(-i-3))
    regressor_Linear = Estimators.KRR(Kernel = LinearKernel, lam = 10**(-i-3))
    regressor_Linear.cross_val(Kernel_train_Linear,Y_train[0],5)

lambda =  0.001


5it [00:03,  1.42it/s]
0it [00:00, ?it/s]

Score :  0.5805
lambda =  0.0001


5it [00:00, 12.85it/s]
0it [00:00, ?it/s]

Score :  0.5860000000000001
lambda =  1e-05


...

Score :  0.596
lambda =  1e-11


5it [00:00, 12.11it/s]
0it [00:00, ?it/s]

Score :  0.5985
lambda =  1e-12


5it [00:00, 11.47it/s]

Score :  0.6005
